In [ ]:
# Import the Libraries : 
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import Input
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import Conv2D, MaxPool2D, add,Dropout,Dense,Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, Callback

import sklearn
from sklearn.metrics import precision_recall_fscore_support
from numpy import save

In [ ]:
# Load the data set to the Google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# we pass the data set into different pakages
mini_batch_size = 700

In [ ]:
!ls "/content/drive/MyDrive/test_source_output"

X_final_STFT_test_source.npy  Y_final_STFT_test_source.npy


In [ ]:
y_train=np.load('/content/drive/MyDrive/Y_final_STFT_.npy',mmap_mode= 'r')
print(y_train.shape)

(32519, 100, 1, 88)


In [ ]:
X_train=np.load('/content/drive/MyDrive/X_final_STFT_.npy',mmap_mode= 'r')
print(X_train.shape)

(32519, 100, 1, 252)


In [ ]:
X_validation=np.load('/content/drive/MyDrive/test_source_output/X_final_STFT_test_source.npy',mmap_mode= 'r')
print(X_validation.shape)

(10351, 100, 1, 252)


In [ ]:
y_validation=np.load('/content/drive/MyDrive/test_source_output/Y_final_STFT_test_source.npy',mmap_mode= 'r')
print(y_validation.shape)

(10351, 100, 1, 88)


In [ ]:
X_test=np.load('/content/drive/MyDrive/test_source_output/X_final_STFT_test_source.npy',mmap_mode= 'r')
print(X_test.shape)

(10351, 100, 1, 252)


In [ ]:
y_test=np.load('/content/drive/MyDrive/test_source_output/Y_final_STFT_test_source.npy', mmap_mode= 'r')
print(y_test.shape)

(10351, 100, 1, 88)


In [ ]:
# The X_train is converted to 4 dimension with a mini_batch size of 400 
X_train = X_train[:int(X_train.shape[0]/mini_batch_size)*mini_batch_size,:]
print(X_train.shape)

(32200, 100, 1, 252)


In [ ]:
# The Y_train is converted to 4 dimension with a mini_batch size of 400 similar to X_train
y_train = y_train[:int(y_train.shape[0]/mini_batch_size)*mini_batch_size,:]
print(y_train.shape)

(32200, 100, 1, 88)


In [ ]:
X_validation = X_validation[:int(X_validation.shape[0]/mini_batch_size)*mini_batch_size,:]
print(X_validation.shape)

(9800, 100, 1, 252)


In [ ]:
y_validation = y_validation[:int(y_validation.shape[0]/mini_batch_size)*mini_batch_size,:]
print(y_validation.shape)

(9800, 100, 1, 88)


In [ ]:
# Shapes of both X_train and y_train)
print(X_train.shape)
print(y_train.shape)

(32200, 100, 1, 252)
(32200, 100, 1, 88)


In [ ]:
# shape of validation set :
print(X_validation.shape)
print(y_validation.shape)

In [ ]:
# preparing test and validation data
train_data = tf.data.Dataset.from_tensor_slices((X_train,y_train))
train_data = train_data.batch(mini_batch_size)

validation_data = tf.data.Dataset.from_tensor_slices((X_validation,y_validation))
validation_data = validation_data.batch(mini_batch_size)

In [ ]:
# parameters for the layers of neural network 
window_size=7
min_midi = 21
max_midi = 108
note_range = max_midi - min_midi + 1

In [ ]:
# layers of Model 
input_shape=(None,100,1,252) #number of samples
model=Sequential()

# 1st Layer 
model.add(Conv2D(filters=32, kernel_size=(5,4),padding="same", activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dropout(rate=0.2))

# 2nd Layer 
model.add(Conv2D(filters=32, kernel_size=(3, 3),padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.2))
model.add(MaxPool2D(pool_size=(1, 2),padding="same"))

#stage3
model.add(Dense(1000, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

# 4th Layer 
model.add(Dense(200, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

# Output Layer
model.add(Dense(88, activation='sigmoid'))

In [ ]:
# Code for the Accuracy of the Model 
class AccuracyHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [ ]:
# Calculating the accuracy of the model using adam as optimizer and the learning rate is 0.001
optimizer1 = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer= optimizer1, metrics=['accuracy'])
history = AccuracyHistory()

# checkpoint = ModelCheckpoint(model_ckpt,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
early_stop = EarlyStopping(patience=10, monitor='val_loss',verbose=1, mode='min')
callbacks = [history]

In [ ]:
model.fit(train_data,epochs=10,batch_size=mini_batch_size,validation_data=validation_data, verbose = 1, callbacks=callbacks)

Epoch 1/10
46/46 [==============================] - 534s 6s/step - loss: 0.6740 - accuracy: 0.0113 - val_loss: 0.5220 - val_accuracy: 0.0034
Epoch 2/10
46/46 [==============================] - 377s 8s/step - loss: 0.3778 - accuracy: 0.0095 - val_loss: 0.2308 - val_accuracy: 0.0093
Epoch 3/10
46/46 [==============================] - 375s 8s/step - loss: 0.2049 - accuracy: 0.0122 - val_loss: 0.1852 - val_accuracy: 0.0219
Epoch 4/10
46/46 [==============================] - 375s 8s/step - loss: 0.1843 - accuracy: 0.0177 - val_loss: 0.1789 - val_accuracy: 0.0219
Epoch 5/10
46/46 [==============================] - 375s 8s/step - loss: 0.1803 - accuracy: 0.0178 - val_loss: 0.1770 - val_accuracy: 0.0219
Epoch 6/10
46/46 [==============================] - 370s 8s/step - loss: 0.1787 - accuracy: 0.0176 - val_loss: 0.1762 - val_accuracy: 0.0219
Epoch 7/10
46/46 [==============================] - 371s 8s/step - loss: 0.1779 - accuracy: 0.0174 - val_loss: 0.1758 - val_accuracy: 0.0219
Epoch 8/10
46

In [ ]:
# Summary of the Model and dimension of different layers
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 100, 1, 32)        161312    
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 100, 1, 32)        128       
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 100, 1, 32)        0         
_________________________________________________________________
module_wrapper_3 (ModuleWrap (None, 100, 1, 32)        9248      
_________________________________________________________________
module_wrapper_4 (ModuleWrap (None, 100, 1, 32)        128       
_________________________________________________________________
module_wrapper_5 (ModuleWrap (None, 100, 1, 32)        0         
_________________________________________________________________
module_wrapper_6 (ModuleWrap (None, 100, 1, 32)        0

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(score)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_true=y_validation
y_scores=model.predict(X_validation)

In [ ]:
def opt_thresholds(y_true,y_scores):
    othresholds = np.zeros(y_scores.shape[1])
    print(othresholds.shape)
    for label, (label_scores, true_bin) in enumerate(zip(y_scores.T,y_true.T)):
        precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true_bin, label_scores)
        max_f1 = 0
        max_f1_threshold = .5
        for r, p, t in zip(recall, precision, thresholds):
            if p + r == 0: continue
            if (2*p*r)/(p + r) > max_f1:
                max_f1 = (2*p*r)/(p + r)
                max_f1_threshold = t
        # print("label %f: max_f1_threshold %f => max_f1 %f" %(label, max_f1_threshold, max_f1))
        othresholds[label] = max_f1_threshold
    return othresholds

In [ ]:
x, y_true=X_validation, y_validation
y_scores=model.predict(x)
y_scores=np.reshape(y_scores, (980000, 88))
y_validation=np.reshape(y_validation, (980000, 88))
thresholds = opt_thresholds(y_true,y_scores)

#evaludate this over validation data
y_pred = y_scores > thresholds
#precision, recall, fbetascore, support =>p, r, f, s
p,r,f,s = sklearn.metrics.precision_recall_fscore_support(y_true,y_pred,average='micro')
print("%f %f %f" %(p, r, f))

(88,)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_ranking.py:677: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


0.081061 0.678433 0.144819


In [ ]:
y_scores_test=model.predict(X_test)
y_pred_test=y_scores_test > thresholds

In [ ]:
print(y_pred_test.shape)

(10351, 100, 1, 88)


In [ ]:
print(thresholds.shape)

In [ ]:
y_pred_test=np.squeeze(y_pred_test, axis=-2)
print(y_pred_test.shape)

(10351, 100, 88)


In [ ]:
#saving the y_pred_test for further post processing and generating piano roll
save('final2.npy', y_pred_test)